<a href="https://colab.research.google.com/github/karyam/rgnn_eeg_emotion_classifier/blob/main/rgnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split
import tensorboard

In [ ]:
SUBJECT_NPY_DATA_PATH = "drive/My Drive/BCI_clone/npy/npy_0.npy"
SUBJECT_NPY_LABEL_PATH = "drive/My Drive/BCI_clone/npy/npy_0_label.npy"

ALL_SUBJECTS_NPY_DATA_PATH = "drive/My Drive/BCI_clone/npy/npy_all_subjects.npy"
ALL_SUBJECTS_NPY_LABEL_PATH = "drive/My Drive/BCI_clone/npy/npy_all_subjects_label.npy"

batch_size = 32
num_nodes = 62
num_features = 25

In [ ]:
# computed in preprocess
min_val = 10.567626836074302
max_val = 42.11366999020901

Build the model input (batched graphs)

In [ ]:
def get_train_data(X, Y, batch_size=32, drop_incomplete=True):
  X_batched, Y_batched = [], []
  num_graphs = X.shape[0]
  
  for i in range(0, num_graphs, batch_size):
    if drop_incomplete is True and (i + batch_size > X.shape[0]): break;
    
    batch = X[i:i+batch_size]
    assert (batch.shape == (batch_size, num_nodes, num_features))
    batch = tf.constant(np.reshape(batch, (-1, num_features)))
    
    assert (batch.shape == (batch_size*num_nodes, num_features))
    assert (tf.math.reduce_min(batch) >= min_val)
    assert (tf.math.reduce_max(batch) <= max_val) 

    X_batched.append(batch)
    Y_batched.append(tf.constant(Y[i:i+batch_size], dtype="float32"))

  X_batched = tf.stack(X_batched, axis=0)
  Y_batched = tf.stack(Y_batched, axis=0)
  assert (X_batched.shape == (num_graphs//batch_size, batch_size*num_nodes, num_features))
  assert (Y_batched.shape == (num_graphs//batch_size, batch_size,))

  return X_batched, Y_batched

In [ ]:
def emotion_dl(batched_labels:list, noise:int=0.18):
  d_labels = []

  def get_distrib(label:int):
    if label == 0: return tf.constant([1-2*noise/3, noise/3, 0.0], dtype="float32")
    if label == 1: return tf.constant([2*noise/3, 1-2*noise/3, 2*noise/3], dtype="float32")
    if label == 2: return tf.constant([0.0, noise/3, 1-2*noise/3], dtype="float32")
  
  for batch in batched_labels:
    batch = tf.map_fn(lambda t: get_distrib(int(t)), batch)
    assert (batch.shape == (batch_size, 3))
    d_labels.append(batch)

  assert (len(d_labels) == len(batched_labels))
  return d_labels

In [ ]:
data_path = SUBJECT_NPY_DATA_PATH
labels_path = SUBJECT_NPY_LABEL_PATH
X = np.load(data_path)
Y = np.load(labels_path)
print(X.shape, Y.shape) #num_graphs x num_ch x num_features
X, Y = get_train_data(X, Y)
X, Y = tf.unstack(X), tf.unstack(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
print(len(X_train))
print(len(X_test))
# get label distributions
y_train, y_test = emotion_dl(y_train), emotion_dl(y_test)

(2025, 62, 25) (2025,)
31
32


In [ ]:
num_batches = min(len(X_train), len(X_test))

In [ ]:
# initialize the adjacency matrix A
# to what extent would help to use a sparse matrix instead?
# A[i][j] = exp(-dist[i][j] / 2*delta^2)
# delta = 0.1
# dist[i][j] - the physical distance between the ith and jth channels
def build_adj_matrix(dist):
  d_shape = dist.shape
  assert (d_shape == (num_nodes,num_nodes))
  A = np.zeros(shape=(num_nodes*batch_size,num_nodes*batch_size))

  for i in range(batch_size):
    for x in range(d_shape[0]):
      for y in range(d_shape[1]):
            A[i*d_shape[0]+x][i*d_shape[1]+y] = dist[x][y]
  # np.save(os.path.join(CSV_PATH, "adj_matrix.npy"), A)
  A = tf.convert_to_tensor(A, dtype="float32")
  return A

In [ ]:
class GCNLayer(tf.keras.layers.Layer):
  '''
  '''
  def __init__(self, hidden_dim=32, activation=tf.keras.activations.relu, 
               activation_att=tf.keras.activations.tanh, pooling_ratio=1):
    super(GCNLayer, self).__init__()
    self.hidden_dim = hidden_dim
    self.activation = activation
    self.activation_att = activation_att
    self.weight_init = tf.keras.initializers.GlorotNormal()
    self.pooling_ratio = pooling_ratio #(0, 1], 1 no pooling

  def get_degree_matrix(self, A:tf.Tensor):
    "sum across second dimension and transform the reduced matrix into eye"
    diagonals = tf.math.reduce_sum(A, axis=1)
    return tf.linalg.tensor_diag(diagonals)
                                 
  def get_norm_laplacian(self, A:tf.Tensor):
    """ Compute L = (D**-1/2) * A * (D**-1/2)
        Make sure A contains self-loops.
    """
    D = self.get_degree_matrix(A)
    D = tf.math.pow(D, -0.5)
    D = tf.where(tf.math.is_inf(D), tf.zeros_like(D), D) # replace inf values with 0
    L = tf.matmul(D, A)
    L = tf.matmul(L, D)
    return L
  
  def topk_batch(self, X, k):
    # reshape to [batch_size, num_nodes]
    X = tf.reshape(X, [batch_size, -1])
    
    # get the topk values from each batch
    topk_values, topk_indices = tf.math.top_k(X, k, sorted=False)

    # add batch no offset 
    offset = tf.constant([[sample]*k for sample in range(batch_size)])
    topk_indices = tf.math.add(topk_indices, offset)

    # flatten to [k*batch_size]
    topk_indices = tf.reshape(topk_indices, [-1])
    topk_values = tf.reshape(topk_values, [-1])
    
    assert(topk_indices.shape == (batch_size*k,))
    assert(topk_values.shape == (batch_size*k,))

    return topk_values, topk_indices
  
  def pool(self, X:tf.Tensor, A):
    # get the self-attention score matrix: X_att = X * W_att
    X_att = self.activation_att(tf.matmul(X, self.W_att)) #shape(N, 1)
    
    # get the nodes with topk attention scores from 
    # each batch; add a treshold to k to be sure k != 0
    k = max(int(np.ceil(self.num_nodes * self.pooling_ratio)), 20)
    topk_values, topk_indices = self.topk_batch(X_att, k)
    
    # apply pooling to node features X = X[topk_indices] * topk_values
    X = tf.gather(X, topk_indices)
    X = tf.transpose(X)
    X = tf.math.multiply(X, topk_values)
    X = tf.transpose(X)

    # filter the adjacency matrix to account for the selected nodes only
    A = tf.gather(A, topk_indices, axis=0)
    A = tf.gather(A, topk_indices, axis=1)

    assert(X.shape==(batch_size*k, self.hidden_dim))
    assert(A.shape==(batch_size*k, batch_size*k))

    return X, A

  def readout(self, X:tf.Tensor):
    """ Readout function to collapse all nodes into a single representation"""
    X = tf.reshape(X, [batch_size,-1,self.hidden_dim])

    global_mean_pool = tf.math.reduce_mean(X, axis=1)
    global_max_pool = tf.math.reduce_max(X, axis=1)

    assert(global_mean_pool.shape == (batch_size, self.hidden_dim))
    assert(global_max_pool.shape == (batch_size, self.hidden_dim))

    return tf.concat([global_mean_pool, global_max_pool], axis=1)

  def build(self, input_shape):
    self.num_nodes = int(input_shape.as_list()[0]) / batch_size
    num_features = input_shape[-1]
    self.W = self.add_weight("W", shape=[num_features, self.hidden_dim], initializer=self.weight_init)
    self.W_att = self.add_weight("W_att", shape=[self.hidden_dim, 1], initializer=self.weight_init)

  def call(self, X:tf.Tensor, A:tf.Tensor):
    ''' 
      Compute graph convolution with self-attention poosing.
      Returns:
        - X: result from graph convolution only
        - X_pooled: result after pooling layer
        - red: readout output
        - A: the adjacency matrix of the pooled graph
    '''
    L = self.get_norm_laplacian(A) #L = D^(-1/2) * A * D^(-1/2)
    X = tf.matmul(L, X) # graph convolution: X = L * X
    X = self.activation(tf.matmul(X, self.W)) #linear embedding: X = sigma(X*W)
    X_pooled, A = self.pool(X, A)  # graph pooling
    red = self.readout(X_pooled) # readout and concatennate
    
    tf.debugging.assert_all_finite(X, "not finite")
    tf.debugging.assert_all_finite(X_pooled, "not finite")
    tf.debugging.assert_all_finite(red, "not finite")
    tf.debugging.assert_all_finite(A, "not finite")
    
    return X, X_pooled, red, A
  

In [ ]:
def init_adj():
  initializer = tf.keras.initializers.RandomUniform(0.5, 0.9)
  values = initializer(shape=(62, 62))
  # replace above with between channels distances
  diagonal = np.array([1]*num_nodes)
  values = tf.linalg.set_diag(values, diagonal)
  A = build_adj_matrix(values)
  A = tf.Variable(A, trainable=True, name="A")
  return A

In [ ]:
class RGNN(tf.keras.Model):
  def __init__(self, num_blocks, out_dim, hidden_dim, A_init=None):
    super(RGNN, self).__init__()
    # hyper-parameters
    self.num_blocks = num_blocks
    self.out_dim = out_dim
    self.hidden_dim = hidden_dim
    
    # the adjacency matrix parameter
    initializer = tf.keras.initializers.GlorotNormal()
    values = initializer(shape=(32*62, 32*62))
    self.A = init_adj()

    self.blocks = [GCNLayer(hidden_dim=hidden_dim, pooling_ratio=0.8) for i in range(num_blocks)]
    # output layers
    self.graph_output = tf.keras.layers.Dense(self.out_dim, activation="softmax", name="target")
    self.domain_output = tf.keras.layers.Dense(2, activation="softmax", name="domain")
  
  def call(self, train, test=None):
    """
    Return:
      - out:tf.Tensor(batch_size, out_dim) - graph-level emotion probabilities
      - domain_train:tf.Tensor(batch_size*num_pooled_nodes, 2)
      - domain_test:tf.Tensor(batch_size*num_pooled_nodes, 2)
    """
    adjS = self.A; adjT = self.A
    redS = []
    convS, pooledS, red, adjS = self.blocks[0](train, adjS)
    convT, pooledT, redT, adjT = self.blocks[0](test, adjT)
    redS.append(red)
    
    for i in range(1, self.num_blocks):
      convS, pooledS, red, adjS = self.blocks[i](pooledS, adjS)
      redS.append(red)
      convT, pooledT, redT, adjT = self.blocks[i](pooledT, adjT)
    
    assert (len(redS) == len(self.blocks))
    # add the readouts from the source domain
    concat_redS = tf.math.add_n(redS)
    
    # node-wise domain classification for finer granularity
    domain_train = self.domain_output(convS)
    domain_test = self.domain_output(convT)
    
    # target task classification logits
    out = self.graph_output(concat_redS)
    return out, domain_train, domain_test

In [ ]:
class Trainer(object):
  """
  Class to train model in EEG classification
  """
  def __init__(self, num_epochs=200, lr=1e-3):
    self.num_epochs = num_epochs
    self.kl = tf.keras.losses.KLDivergence()
    self.dl = tf.keras.losses.SparseCategoricalCrossentropy()
    self.optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    self.beta = 0
    self.model = RGNN(num_blocks=3, out_dim=3, hidden_dim=64)

  def run(self):
    for epoch in range(self.num_epochs):
      epoch_loss_avg = tf.keras.metrics.Mean()
      for i in range(num_batches):
        graph_train = X_train[i]
        graph_test = X_test[i]
        labels_train = tf.constant(y_train[i])
        
        assert (graph_train.shape == (batch_size*num_nodes, num_features))
        assert (graph_test.shape == (batch_size*num_nodes, num_features))
        assert (labels_train.shape == (batch_size, 3))
        
        with tf.GradientTape(persistent=True) as tape:
          graph_preds, domain_train, domain_test = self.model(graph_train, graph_test)
          domain_preds = tf.concat([domain_train, domain_test], axis=0)
          domain_labels = tf.concat([tf.convert_to_tensor([0]*domain_train.shape[0]),
                                     tf.convert_to_tensor([1]*domain_test.shape[0])], axis=0) 
          assert (domain_preds.shape == (2*domain_train.shape[0], 2))
          assert (domain_labels.shape == (domain_preds.shape[0],))
      
          target_loss = self.kl(labels_train, graph_preds)
          domain_loss = self.dl(domain_labels, domain_preds)
      
        # params order: gcn1/W, gcn1/W_att, gcn2/W, gcn2/W_att, gcn3/W, gcn3/W_att, W_out, bias_out W_domain, bias_domain, A
        print(f'Loss: {target_loss}')
        params = self.model.trainable_variables
        # print(len(params))
        # for param in params:
        #   print(param.name)
        # print(f'domain_loss:{domain_loss}')
        
        target_grads = tape.gradient(target_loss, params[0:-3]) + [tape.gradient(target_loss, params[-1])]
        reverse_grads = tape.gradient(domain_loss, params[0:-6]) + [tape.gradient(domain_loss, params[-1])]
        domain_grads = tape.gradient(domain_loss, params[-3:-1]) 

        for i, grad in enumerate(reverse_grads): 
          if grad is None: print(i)

        for grad in target_grads: tf.debugging.assert_all_finite(grad, "not finite target grad")
        for grad in reverse_grads: tf.debugging.assert_all_finite(grad, "not finite reverse grad")
        for grad in domain_grads: tf.debugging.assert_all_finite(grad, "not finite domain grad")
        
        # minimize the KL loss for the target task
        self.optimizer.apply_gradients(zip(target_grads, params[0:-3]+[params[-1]]))
        
        # minimize the domain loss with respect to the params of the domain_output layer
        self.optimizer.apply_gradients(zip(domain_grads, params[-3:-1]))

        # negate the gradients and multiply by annealed factor beta
        reverse_grads = [tf.math.negative(reverse_grads[i]) for i in range(len(reverse_grads))]
        progress = (epoch+1) / self.num_epochs
        self.beta = 2/(1 + np.exp(-10*progress)) - 1
        reverse_grads = [tf.math.scalar_mul(self.beta, reverse_grads[i]) for i in range(len(reverse_grads))] 
        
        # maximize the domain loss with respect to the rest of params
        self.optimizer.apply_gradients(zip(reverse_grads, params[0:-6]+[params[-1]]))
        epoch_loss_avg.update_state(target_loss)

      print("Epoch: {:03d}, Loss: {:.3f}". format(epoch, epoch_loss_avg.result()))
    

In [ ]:
trainer = Trainer()
trainer.run()